<a href="https://colab.research.google.com/github/slsabilaAura/Tugas-Akhir/blob/main/Model_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gensim

In [2]:
# prompt: import file form drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd

proposal_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_proposalC.csv')
expert_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_expertC.csv')


In [4]:
import numpy as np
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models
from gensim.matutils import corpus2dense
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.matutils import corpus2dense
from gensim.matutils import sparse2full
import pandas as pd
import ast

# Corpus dan Dictionary

In [31]:
def convert_to_list(text):
    try:
        return ast.literal_eval(text) if isinstance(text, str) else text
    except:
        return []


proposal_df["processed"] = proposal_df["stemmed"].apply(convert_to_list)
expert_df["processed"] = expert_df["stemmed"].apply(convert_to_list)

proposal_df["processed"] = proposal_df["processed"].apply(lambda x: x[:30])
expert_df["processed"] = expert_df["processed"].apply(lambda x: x[:30])

# Gabungkan semua dokumen untuk membuat satu kamus bersama
documents_all = proposal_df["processed"].tolist() + expert_df["processed"].tolist()
dictionary_all = Dictionary(documents_all)


dictionary_all.filter_extremes(no_below=5, no_above=0.5)

# Buat corpus (Bag of Words)
corpus_combined = [dictionary_all.doc2bow(text) for text in documents_all]

# Buat model TF-IDF
tfidf_model = models.TfidfModel(corpus_combined)

# Transformasikan corpus ke TF-IDF
tfidf_corpus_combined = tfidf_model[corpus_combined]

n_proposals = len(proposal_df)
n_experts = len(expert_df)

tfidf_proposal = tfidf_corpus_combined[:n_proposals]
tfidf_expert = tfidf_corpus_combined[n_proposals:]

# Buat matriks dense (dokumen x vocab_size)
proposal_dense = corpus2dense(tfidf_proposal, num_terms=len(dictionary_all)).T
expert_dense = corpus2dense(tfidf_expert, num_terms=len(dictionary_all)).T

In [32]:
import pandas as pd

# Ambil daftar dokumen dari tfidf_corpus_combined
tfidf_data = []

for doc_index, doc in enumerate(tfidf_corpus_combined):
    for term_id, weight in doc:
        tfidf_data.append({
            "doc_id": doc_index,
            "term": dictionary_all[term_id],
            "term_id": term_id,
            "tfidf": weight
        })

# Ubah ke DataFrame
tfidf_df = pd.DataFrame(tfidf_data)

doc1_tfidf = tfidf_df[tfidf_df["doc_id"] == 1]
print(doc1_tfidf.sort_values(by="tfidf", ascending=False))  # Urut dari tertinggi



    doc_id      term  term_id     tfidf
14       1     cepat       14  0.431814
24       1     pikir       24  0.418914
25       1      pola       25  0.328399
23       1      maju       23  0.324193
21       1      kota       21  0.223962
29       1  sekarang       29  0.209457
28       1    segala       28  0.209457
16       1      desa       16  0.198623
26       1    sangat       26  0.194969
17       1       era       17  0.168742
31       1     tidak       31  0.155252
22       1      luas       22  0.152889
19       1     jalan       19  0.146720
27       1     sebar       27  0.142611
30       1    sosial       30  0.140075
18       1     hanya       18  0.134318
13       1    bidang       13  0.133254
15       1    dampak       15  0.133254
12       1      baik       12  0.108979
20       1   kembang       20  0.082849


# Directed M->D

In [33]:
import pandas as pd

# tfidf_df_proposal_contoh = pd.DataFrame(proposal_dense, columns=[f"term_{i}" for i in range(proposal_dense.shape[1])])
# print(tfidf_df_proposal_contoh.head())
terms = [dictionary_all[i] for i in range(len(dictionary_all))]
tfidf_df_proposal_contoh = pd.DataFrame(proposal_dense, columns=terms)
# print(tfidf_df_proposal_contoh)


In [34]:
# Konversi ke DataFrame
proposal_df_tfidf = pd.DataFrame(proposal_dense)
expert_df_tfidf = pd.DataFrame(expert_dense)

# Simpan ke CSV
# proposal_df_tfidf.to_csv('/content/drive/MyDrive/Skripsi4/kata/hasil_TFIDF_proposal_lama.csv', index=False)
# expert_df_tfidf.to_csv('/content/drive/MyDrive/Skripsi4/kata/hasil_TFIDF_expert_lama.csv', index=False)

In [35]:
proposal_df["tahun"] = proposal_df["proposal_year"].astype(int)
expert_df["pub_year"] = expert_df["research_pub_year"].astype(int)


def similarity_m_to_d(proposal_vector, expert_vector):
    numerator = np.dot(proposal_vector, expert_vector)
    denominator = (np.linalg.norm(proposal_vector) + 1e-10)
    return numerator / (denominator)


def similarity_d_to_m(expert_vector, proposal_vector):
    numerator = np.dot(expert_vector, proposal_vector)
    denominator = (np.linalg.norm(expert_vector) + 1e-10)
    return numerator / (denominator)

 # denominator = np.linalg.norm(expert_vector)

def time_decay(year_prop, year_ex, t=1, gamma=0.1):
    decay = 1 - ((year_prop - year_ex) / t) * gamma
    return max(decay, 0.0)  # tidak boleh negatif

# mapping Dosen dengan ID Dosen

dosen_id_map = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/mapping.csv")  # pastikan kolom: expert_id, expert_name
dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()


def attach_dosen_id(df, dosen_id_map):

    df = df.copy()
    df["dosen"] = df["dosen"].str.strip().str.lower()  # normalisasi agar cocok
    dosen_id_map = dosen_id_map.copy()
    dosen_id_map["dosen"] = dosen_id_map["dosen"].str.strip().str.lower()

    merged_df = df.merge(dosen_id_map, on="dosen", how="left")
    return merged_df



def explode_authors_with_weights(df, dosen_id_map):
    rows = []

    # Normalisasi nama dosen agar cocok
    dosen_id_map = dosen_id_map.copy()
    dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()

    for _, row in df.iterrows():
        authors = row.get("authors", [])
        # Bersihkan nama kosong atau NaN
        authors = [a for a in authors if isinstance(a, str) and a.strip() != ""]

        num_authors = len(authors)
        for idx, name in enumerate(authors):
            name_clean = name.strip().lower()

            if num_authors == 1:
                weight = 1.0
            else:
                weight = 0.6 if idx == 0 else 0.4 / (num_authors - 1)

            new_row = row.to_dict()
            new_row["name"] = name
            new_row["author_position"] = idx + 1
            new_row["num_authors"] = num_authors
            new_row["author_weight"] = round(weight, 4)

            matched = dosen_id_map[dosen_id_map["expert_name"] == name_clean]
            new_row["expert_id"] = matched["expert_id"].values[0] if not matched.empty else None

            rows.append(new_row)

    return pd.DataFrame(rows)


# Gabungkan author_1 sampai author_6 jadi list
author_cols = ["author_1", "author_2", "author_3", "author_4", "author_5", "author_6"]
expert_df["authors"] = expert_df[author_cols].values.tolist()

# Hapus duplikat berdasarkan research_id
expert_df = expert_df.drop_duplicates(subset=["research_id"]).copy()

# Jalankan explode
expert_df = explode_authors_with_weights(expert_df, dosen_id_map)

# Opsional: hanya simpan baris dengan expert_id valid
expert_df = expert_df[expert_df["expert_id"].notna()]



# expert_df.to_csv("/content/drive/MyDrive/Skripsi4/dictionary/expert_df_id_dosen.csv", index=False)

## OD M->D

In [36]:
def compute_od_m_to_d(tfidf_proposal, tfidf_expert, proposal_df, expert_df):
    records = []

    for idx_prop, prop_row in proposal_df.iterrows():
        prop_id = prop_row["proposal_id"]
        prop_vector = tfidf_proposal[idx_prop]
        prop_year = prop_row["tahun"]
        mahasiswa = prop_row["student_id"]

        for idx_expert, exp_row in expert_df.iterrows():
            exp_id_dosen = exp_row["expert_id"]
            author_position = exp_row["author_position"]
            author_weight = exp_row["author_weight"]
            pub_year = exp_row["pub_year"]
            penelitian_id = exp_row["research_id"]
            dosen = exp_row["name"]
            exp_vector = tfidf_expert[idx_expert]

            sim_m2d = similarity_m_to_d(prop_vector, exp_vector)
            od_score = sim_m2d * author_weight

            records.append({
                "id_proposal": prop_id,
                "mahasiswa": mahasiswa,
                "penelitian_id": penelitian_id,
                "id_dosen": exp_id_dosen,
                "dosen": dosen,
                "author_position": author_position,
                "tahun_proposal": prop_year,
                "tahun_penelitian": pub_year,
                "author_weight": author_weight,
                "OD(M→D)": round(od_score, 4)
            })

    df_scores = pd.DataFrame(records)

    # Ambil OD tertinggi untuk tiap pasangan proposal-dosen
    df_scores['id_proposal'] = df_scores['id_proposal'].astype(str)
    # df_scores = df_scores.loc[df_scores.groupby(["id_proposal", "id_dosen"])["OD(M→D)"].idxmax()]
    df_scores = df_scores[df_scores["OD(M→D)"] > 0.02]
    df_scores = df_scores.reset_index(drop=True)

    # Urutkan
    df_scores = df_scores.sort_values(by=["id_proposal", "OD(M→D)"], ascending=[True, False])

    return df_scores

# Hitung ulang
df_od = compute_od_m_to_d(
    proposal_dense,
    expert_dense,
    proposal_df,
    expert_df
)
df_od = df_od[df_od["OD(M→D)"] > 0.02]

print(df_od[["id_proposal", "mahasiswa", "id_dosen", "dosen", "penelitian_id", "author_position", "author_weight","OD(M→D)"]].head())







# # Lihat hasil
print(df_od[["id_proposal","mahasiswa", "id_dosen", "dosen", "author_position", "penelitian_id", "tahun_proposal","tahun_penelitian", "OD(M→D)"]].head(5))
# df_od.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/od_m2d_kata.csv", index=False)

    id_proposal mahasiswa id_dosen            dosen penelitian_id  \
99           P1        S1       D1  Bambang Harjito          R456   
72           P1        S1       D1  Bambang Harjito          R345   
3            P1        S1       D1  Bambang Harjito           R32   
100          P1        S1      D15    Heri Prasetyo          R456   
40           P1        S1      D15    Heri Prasetyo          R186   

     author_position  author_weight  OD(M→D)  
99                 1            0.6   0.2402  
72                 1            0.6   0.1714  
3                  1            1.0   0.1683  
100                2            0.4   0.1601  
40                 1            1.0   0.1294  
    id_proposal mahasiswa id_dosen            dosen  author_position  \
99           P1        S1       D1  Bambang Harjito                1   
72           P1        S1       D1  Bambang Harjito                1   
3            P1        S1       D1  Bambang Harjito                1   
100          P1

## TOD M->D

In [37]:
def compute_tod_m2d(od_df, t=1, gamma=0.1, min_year_diff=0, max_year_diff=5):
    od_df = od_df.copy()

    # Hitung selisih tahun
    od_df["selisih_tahun"] = od_df["tahun_proposal"] - od_df["tahun_penelitian"]

    # Filter: selisih tahun antara 3 dan 5 tahun
    od_df = od_df[
        (od_df["selisih_tahun"] >= min_year_diff) &
        (od_df["selisih_tahun"] <= max_year_diff)
    ].copy()

    # Hitung time decay factor
    od_df["time_decay_factor"] = od_df.apply(
        lambda row: time_decay(row["tahun_proposal"], row["tahun_penelitian"], t=t, gamma=gamma),
        axis=1
    )

    # Hitung TOD(M→D)
    od_df["TOD(M→D)"] = (od_df["OD(M→D)"] * od_df["time_decay_factor"]).round(4)

    return od_df

# Panggil fungsi dengan batas selisih tahun antara 3–5
tod_m2d_df = compute_tod_m2d(df_od, t=1, gamma=0.1, min_year_diff=0, max_year_diff=5)
# Filter nilai TOD(M→D) > 0.04
# tod_m2d_df = tod_m2d_df[tod_m2d_df["TOD(M→D)"] > 0.06]

# Urutkan berdasarkan skor rekomendasi
tod_m2d_df = (
    tod_m2d_df
    .loc[tod_m2d_df.groupby(["id_proposal", "id_dosen"])["TOD(M→D)"].idxmax()]
    .reset_index(drop=True)
    .sort_values(by=["id_proposal", "TOD(M→D)"], ascending=[True, False])
)

# Tampilkan kolom penting
print(tod_m2d_df[[
    "id_proposal", "mahasiswa", "penelitian_id", "dosen", "id_dosen",
    "author_position", "author_weight", "OD(M→D)", "tahun_penelitian",
    "tahun_proposal", "selisih_tahun", "time_decay_factor", "TOD(M→D)"
]].head(5))

# Simpan ke CSV
# tod_m2d_df.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/tod_m2d_df_kata.csv", index=False)


   id_proposal mahasiswa penelitian_id            dosen id_dosen  \
4           P1        S1          R186    Heri Prasetyo      D15   
0           P1        S1           R48  Bambang Harjito       D1   
2           P1        S1          R291          Wiharto      D12   
11          P1        S1           R48     Esti Suryani       D8   
7           P1        S1          R264      Umi Salamah       D3   

    author_position  author_weight  OD(M→D)  tahun_penelitian  tahun_proposal  \
4                 1            1.0   0.1294              2020            2021   
0                 1            0.6   0.0882              2018            2021   
2                 1            1.0   0.0891              2017            2021   
11                2            0.4   0.0588              2018            2021   
7                 1            1.0   0.0809              2016            2021   

    selisih_tahun  time_decay_factor  TOD(M→D)  
4               1                0.9    0.1165  
0     

## Directed D→M

In [38]:

def compute_od_d_to_m(tfidf_expert, tfidf_proposal, proposal_df, expert_df):
    records = []


    for idx_expert, exp_row in expert_df.iterrows():
        exp_id_dosen = exp_row["expert_id"]
        author_position = exp_row["author_position"]
        author_weight = exp_row["author_weight"]
        pub_year = exp_row["pub_year"]
        penelitian_id = exp_row["research_id"]
        dosen = exp_row["name"]
        exp_vector = tfidf_expert[idx_expert]

        for idx_prop, prop_row in proposal_df.iterrows():
            prop_id = prop_row["proposal_id"]
            prop_vector = tfidf_proposal[idx_prop]
            prop_year = prop_row["tahun"]
            mahasiswa=prop_row["student_id"]

            selisih_tahun = prop_year - pub_year
            if 0 <= selisih_tahun <= 5 & pub_year <= prop_year:
              sim_d2m = similarity_d_to_m(exp_vector, prop_vector)
              od_score = sim_d2m * author_weight

              records.append({
                  "id_proposal": prop_id,
                  "penelitian_id": penelitian_id,
                  "id_dosen": exp_id_dosen,
                  'mahasiswa': mahasiswa,
                  "dosen": dosen,
                  "author_position": author_position,
                  "author_weight": author_weight,
                  "tahun_penelitian": pub_year,
                  "tahun_proposal": prop_year,
                  "OD(D→M)": round(od_score, 4)
              })

    df_scores = pd.DataFrame(records)
    df_scores = df_scores.loc[df_scores.groupby(["id_proposal", "id_dosen"])["OD(D→M)"].idxmax()].reset_index(drop=True)
    df_scores['id_proposal'] = df_scores['id_proposal'].astype(str)
    df_scores = df_scores.sort_values(by=["id_dosen", "OD(D→M)"], ascending=[True, False])
    return df_scores

df_od_d2m = compute_od_d_to_m(
    expert_dense,
    proposal_dense,
    proposal_df, expert_df
)

# df_od_d2m = df_od_d2m[df_od_d2m["OD(D→M)"] > 0.05]


print(df_od_d2m[["id_proposal","mahasiswa", "id_dosen", "dosen", "author_position", "author_weight", "penelitian_id",
                  "tahun_proposal", "tahun_penelitian", "OD(D→M)"]].head(5))

# df_od_d2m.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/df_od_d2m_kata.csv", index=False)


     id_proposal mahasiswa id_dosen            dosen  author_position  \
186          P11       S11       D1  Bambang Harjito                1   
602         P134      S134       D1  Bambang Harjito                1   
264         P114      S114       D1  Bambang Harjito                1   
1296         P47       S47       D1  Bambang Harjito                1   
248         P113      S113       D1  Bambang Harjito                1   

      author_weight penelitian_id  tahun_proposal  tahun_penelitian  OD(D→M)  
186             1.0          R253            2023              2023   0.3273  
602             1.0           R54            2021              2021   0.3008  
264             1.0           R54            2023              2021   0.2761  
1296            0.6          R364            2023              2023   0.2729  
248             1.0          R363            2023              2023   0.2119  


# Rekomendasi

In [39]:
def combine_overlap_scores(df_m2d, df_d2m, top_n=17):
    # Ambil Top-N dari masing-masing arah
    top_m2d = df_m2d.groupby("id_proposal").head(top_n)[["id_proposal", "id_dosen","mahasiswa", "dosen", "TOD(M→D)"]]
    top_d2m = df_d2m.groupby("id_proposal").head(top_n)[["id_proposal", "id_dosen","mahasiswa", "dosen","OD(D→M)"]]

    # Outer join untuk semua pasangan top-n
    merged = pd.merge(top_m2d, top_d2m, on=["id_proposal", "id_dosen"], how="outer")

    # Tambahkan kolom nama dosen jika hilang (dari M→D arah saja)
    if "dosen_x" in merged.columns:
        merged["dosen"] = merged["dosen_x"].combine_first(merged.get("dosen_y"))
    elif "dosen" not in merged.columns:
        merged["dosen"] = None

    if "mahasiswa_x" in merged.columns:
        merged["mahasiswa"] = merged["mahasiswa_x"].combine_first(merged.get("mahasiswa_y"))
    elif "mahasiswa" not in merged.columns:
        merged["mahasiswa"] = None

    # Ganti NaN skor dengan 0 agar bisa dihitung rata-ratanya
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Tandai overlap jika dosen muncul di kedua arah
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Hitung skor rata-rata (hanya jika overlap)
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

    # Ambil skor tertinggi per proposal dan dosen
    final_scores = merged[["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)","OD(D→M)", "skor_rata2", "overlap"]]
    final_scores = final_scores.sort_values(by=["id_proposal", "skor_rata2"], ascending=[True, False])

    return final_scores

df_final = combine_overlap_scores(tod_m2d_df, df_od_d2m, top_n=17)
print(df_final.head(5))

# df_final.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/overlap_kombinasi_kata.csv", index=False)

   id_proposal mahasiswa               dosen id_dosen  TOD(M→D)  OD(D→M)  \
5           P1        S1       Heri Prasetyo      D15    0.1165   0.1294   
0           P1        S1     Bambang Harjito       D1    0.0617   0.0376   
9           P1        S1  Dewi Wisnu Wardani       D4    0.0394   0.0394   
3           P1        S1             Wiharto      D12    0.0535   0.0190   
13          P1        S1        Esti Suryani       D8    0.0412   0.0286   

    skor_rata2  overlap  
5      0.12295     True  
0      0.04965     True  
9      0.03940     True  
3      0.03625     True  
13     0.03490     True  


# Rangking Directed

In [40]:
def combine_overlap_scores_with_ranking(df_m2d, df_d2m):
    # Ambil semua skor dari kedua arah
    m2d = df_m2d[["id_proposal", "id_dosen","mahasiswa", "dosen", "TOD(M→D)"]]
    d2m = df_d2m[["id_proposal","id_dosen","mahasiswa","dosen", "OD(D→M)"]]

    # Outer join agar semua kombinasi muncul
    merged = pd.merge(m2d, d2m, on=["id_proposal", "id_dosen"], how="outer")

    # Tambahkan kolom nama dosen jika hilang (dari M→D arah saja)
    if "dosen_x" in merged.columns:
        merged["dosen"] = merged["dosen_x"].combine_first(merged.get("dosen_y"))
    elif "dosen" not in merged.columns:
        merged["dosen"] = None

    if "mahasiswa_x" in merged.columns:
        merged["mahasiswa"] = merged["mahasiswa_x"].combine_first(merged.get("mahasiswa_y"))
    elif "mahasiswa" not in merged.columns:
        merged["mahasiswa"] = None

    # Isi nilai NaN dengan 0 untuk penggabungan skor
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Overlap = muncul di kedua arah
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Skor rata-rata jika overlap
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

       # Hitung ranking per proposal berdasarkan skor rata-rata (tanpa groupby + agg)
    merged["rank"] = merged.groupby("id_proposal")["skor_rata2"]\
                           .rank(ascending=False, method="dense")\
                           .astype(int)

    # Ambil kolom yang diinginkan dan urutkan
    result = merged.sort_values(by=["id_proposal", "rank"])[
        ["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap", "rank"]
    ]


    return result


df_peringkat = combine_overlap_scores_with_ranking(tod_m2d_df, df_od_d2m)
# Filter hanya yang overlap == True
df_overlap_true = df_peringkat[df_peringkat["overlap"] == True]
print(df_overlap_true.head(20))

# df_overlap_true.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/rank_overlap_true_kata.csv", index=False)


   id_proposal mahasiswa               dosen id_dosen  TOD(M→D)  OD(D→M)  \
5           P1        S1       Heri Prasetyo      D15    0.1165   0.1294   
0           P1        S1     Bambang Harjito       D1    0.0617   0.0376   
9           P1        S1  Dewi Wisnu Wardani       D4    0.0394   0.0394   
3           P1        S1             Wiharto      D12    0.0535   0.0190   
13          P1        S1        Esti Suryani       D8    0.0412   0.0286   
8           P1        S1         Umi Salamah       D3    0.0404   0.0286   
2           P1        S1             Winarno      D11    0.0319   0.0354   
7           P1        S1             Wiranto       D2    0.0309   0.0309   
6           P1        S1     Ardhi Wijayanto      D16    0.0215   0.0179   
4           P1        S1     Haryono Setiadi      D14    0.0196   0.0192   
14          P1        S1    Sari Widya Sihwi       D9    0.0241   0.0106   
23         P10       S10             Wiranto       D2    0.2004   0.2227   
15         P

## Beban Kerja

In [41]:
from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count_directed = defaultdict(int)
final_assignment_directed = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in df_overlap_true["id_proposal"].unique():
    candidates = df_overlap_true[df_overlap_true["id_proposal"] == pid]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count_directed[dosen_id] < 15:
            rank1_count_directed[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count_directed[dosen_id]
            final_assignment_directed.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count_directed[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count_directed[dosen_id]
        final_assignment_directed.append(row)

# 3. Buat dataframe dari rank 1
rank1_directed = pd.DataFrame(final_assignment_directed)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in df_overlap_true["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_directed[rank1_directed["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = df_overlap_true[(df_overlap_true["id_proposal"] == pid) & (~df_overlap_true["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count_directed[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
final_df = pd.concat([rank1_directed, pd.DataFrame(other_ranks)], ignore_index=True)
final_df = final_df.sort_values(by=["id_proposal", "rank"])

# 🔟 Filter hanya Top 10 dosen per proposal
final_df = final_df[final_df["rank"] <= 17]



# 6. Tampilkan hasil
print(final_df[["id_proposal", "mahasiswa", "id_dosen", "dosen", "skor_rata2", "rank", "beban"]])

# final_df.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/beban_kerja_kata.csv", index=False)

     id_proposal mahasiswa id_dosen               dosen  skor_rata2  rank  \
0             P1        S1      D15       Heri Prasetyo     0.12295     1   
142           P1        S1       D1     Bambang Harjito     0.04965     2   
143           P1        S1       D4  Dewi Wisnu Wardani     0.03940     3   
144           P1        S1      D12             Wiharto     0.03625     4   
145           P1        S1       D8        Esti Suryani     0.03490     5   
...          ...       ...      ...                 ...         ...   ...   
2119         P99       S99       D7      Wisnu Widiarto     0.05920    13   
2120         P99       S99      D11             Winarno     0.05770    14   
2121         P99       S99      D13   Hasan Dwi Cahyono     0.04435    15   
2122         P99       S99       D5          Abdul Aziz     0.03265    16   
2123         P99       S99       D6       Ristu Saptono     0.01765    17   

      beban  
0         1  
142      13  
143      14  
144      15  
145  

# Evaluasi Directed

In [42]:
from sklearn.metrics import precision_score, recall_score

# Load the true labels DataFrame
true_label_df = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/true_labels.csv")


# Ubah kolom author, author2, author3 menjadi lowercase
true_label_df["author1"] = true_label_df["examiner_1"].astype(str).str.strip()
true_label_df["author2"] = true_label_df["examiner_2"].astype(str).str.strip()
true_label_df["author3"] = true_label_df["examiner_3"].astype(str).str.strip()



## Evaluasi Baru

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


def evaluate_ordered_recommendation_cosine(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
        rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df = evaluate_ordered_recommendation_cosine(final_df, true_label_df)
print(result_df)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.335681              0.260563              0.098592   
1      5               0.462441              0.260563              0.098592   
2      7               0.577465              0.260563              0.098592   
3     10               0.723005              0.260563              0.098592   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0               0.06338                  0.754492  
1               0.06338                  0.695801  
2               0.06338                  0.630672  
3               0.06338                  0.542393  


In [44]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_directed(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
    rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_directed_3 = evaluate_per_proposal_directed(final_df, true_label_df, top_n=3)
eval_per_proposal_directed_5 = evaluate_per_proposal_directed(final_df, true_label_df, top_n=5)
eval_per_proposal_directed_7 = evaluate_per_proposal_directed(final_df, true_label_df, top_n=7)
eval_per_proposal_directed_10 = evaluate_per_proposal_directed(final_df, true_label_df, top_n=10)

# eval_per_proposal_directed_3.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/hasil_eval_3_kata.csv", index=False)
# eval_per_proposal_directed_5.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/hasil_eval_5_kata.csv", index=False)
# eval_per_proposal_directed_7.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/hasil_eval_7_kata.csv", index=False)
# eval_per_proposal_directed_10.to_csv("/content/drive/MyDrive/Skripsi4/kata/directed/hasil_eval_10_kata.csv", index=False)

# Cosine

In [45]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Hitung similarity
similarity_matrix = cosine_similarity(proposal_dense, expert_dense)

# Konversi ke DataFrame
similarity_df = pd.DataFrame(similarity_matrix)

print(similarity_df)

# Simpan ke CSV
# similarity_df.to_csv("/content/drive/MyDrive/Skripsi4/dictionary/similarity_MATRIX_kata_.csv", index=False)


          0         1         2         3         4         5         6    \
0    0.025885  0.025885  0.000000  0.016640  0.016640  0.007060  0.007060   
1    0.000000  0.000000  0.000000  0.034183  0.034183  0.019044  0.019044   
2    0.006330  0.006330  0.043849  0.029157  0.029157  0.011413  0.011413   
3    0.000000  0.000000  0.000000  0.036061  0.036061  0.012357  0.012357   
4    0.099009  0.099009  0.008426  0.012672  0.012672  0.020999  0.020999   
..        ...       ...       ...       ...       ...       ...       ...   
137  0.032069  0.032069  0.000000  0.000000  0.000000  0.009633  0.009633   
138  0.014635  0.014635  0.037684  0.018834  0.018834  0.006468  0.006468   
139  0.055982  0.055982  0.000000  0.000000  0.000000  0.019167  0.019167   
140  0.049723  0.049723  0.000000  0.000000  0.000000  0.052787  0.052787   
141  0.048280  0.048280  0.003714  0.006355  0.006355  0.009018  0.009018   

          7         8         9    ...       750       751       752  \
0  

In [46]:
# Filter sesuai kondisi
similarity_cosine_df = []

for i, (_, mahasiswa) in enumerate(proposal_df.iterrows()):
    for j, (_, dosen) in enumerate(expert_df.iterrows()):
        # weight = dosen.get("author_weight", 1.0)
        # score_akhir = similarity_matrix[i, j] * weight
        # score_akhir = similarity_matrix[i, j]
        similarity_cosine_df.append({
            "id_proposal": mahasiswa["proposal_id"],
            "mahasiswa": mahasiswa["student_id"],
            "id_dosen": dosen["expert_id"],
            "id_penelitian": dosen["research_id"],
            "tahun_proposal": mahasiswa["tahun"],
            "tahun_penelitian": dosen["pub_year"],
            "selisih_tahun": mahasiswa["tahun"] - dosen["pub_year"],
            "dosen": dosen["name"],
           "author_position": dosen["author_position"],
            # "weight": weight,
            "similarity_score" : similarity_matrix[i, j],
            # "similarity_score_akhir": score_akhir,
        })

similarity_cosine_df = pd.DataFrame(similarity_cosine_df)


similarity_cosine = similarity_cosine_df[
    (similarity_cosine_df["tahun_proposal"] > similarity_cosine_df["tahun_penelitian"]) &
    (similarity_cosine_df["selisih_tahun"] > 0) &
    (similarity_cosine_df["selisih_tahun"] <= 5)
].copy()


# similarity_cosine.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/similarity_cosine_kata_baru_sbelumfilter.csv", index=False)




In [47]:
# Ambil baris dengan similarity tertinggi untuk kombinasi unik id_proposal dan id_dosen
idx = similarity_cosine.groupby(["id_proposal", "id_dosen"])["similarity_score"].idxmax()
similarity_cosine = similarity_cosine.loc[idx]

# Ranking ulang berdasarkan proposal
similarity_cosine["rank"] = similarity_cosine.groupby("id_proposal")["similarity_score"] \
                                     .rank(method="first", ascending=False).astype(int)

# Urutkan
similarity_cosine = similarity_cosine.sort_values(["id_proposal", "rank"])
similarity_cosine = similarity_cosine[similarity_cosine["rank"] <= 17]


# Tampilkan
print(similarity_cosine[["id_proposal", "mahasiswa","tahun_proposal","id_penelitian", "tahun_penelitian", "id_dosen", "dosen","selisih_tahun", "author_position", "similarity_score","rank"]])
# similarity_cosine.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/similarity_cosine_kata_baru_setelah filter.csv", index=False)


      id_proposal mahasiswa  tahun_proposal id_penelitian  tahun_penelitian  \
83             P1        S1            2021           R48              2018   
84             P1        S1            2021           R48              2018   
305            P1        S1            2021          R186              2020   
470            P1        S1            2021          R291              2017   
434            P1        S1            2021          R264              2016   
...           ...       ...             ...           ...               ...   
75057         P99       S99            2024          R364              2023   
75082         P99       S99            2024          R377              2023   
75236         P99       S99            2024          R464              2022   
74888         P99       S99            2024          R247              2019   
74574         P99       S99            2024           R55              2021   

      id_dosen              dosen  selisih_tahun  a

In [48]:
from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count = defaultdict(int)
final_assignments = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in similarity_cosine["id_proposal"].unique():
    candidates = similarity_cosine[similarity_cosine["id_proposal"] == pid]
    candidates = candidates.sort_values(by="similarity_score", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count[dosen_id] < 15:
            rank1_count[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count[dosen_id]
            final_assignments.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count[dosen_id]
        final_assignments.append(row)

# 3. Buat dataframe dari rank 1
rank1_df = pd.DataFrame(final_assignments)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in similarity_cosine["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_df[rank1_df["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = similarity_cosine[(similarity_cosine["id_proposal"] == pid) & (~similarity_cosine["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="similarity_score", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
rank_df = pd.concat([rank1_df, pd.DataFrame(other_ranks)], ignore_index=True)
rank_df = rank_df.sort_values(by=["id_proposal", "rank"])

# 🔟 Filter hanya Top 10 dosen per proposal
rank_all_df = rank_df[rank_df["rank"] <= 17]



# 6. Tampilkan hasil
print(rank_all_df[["id_proposal", "mahasiswa", "id_dosen", "dosen", "similarity_score", "rank", "beban"]])
# rank_all_df.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/beban_kata.csv", index=False)


     id_proposal mahasiswa id_dosen              dosen  similarity_score  \
0             P1        S1       D1    Bambang Harjito          0.146956   
142           P1        S1       D8       Esti Suryani          0.146956   
143           P1        S1      D15      Heri Prasetyo          0.129410   
144           P1        S1      D12            Wiharto          0.089078   
145           P1        S1       D3        Umi Salamah          0.080904   
...          ...       ...      ...                ...               ...   
2243         P99       S99      D11            Winarno          0.098296   
2244         P99       S99      D13  Hasan Dwi Cahyono          0.085260   
2245         P99       S99      D15      Heri Prasetyo          0.083299   
2246         P99       S99       D6      Ristu Saptono          0.064512   
2247         P99       S99       D5         Abdul Aziz          0.064071   

      rank  beban  
0        1      1  
142      2      4  
143      3     15  
144    

In [49]:
# Urutkan berdasarkan id_proposal, similarity_score_akhir (descending), dan author_position (ascending)
similarity_cosine_df_baru = rank_all_df.sort_values(
    by=["id_proposal", "similarity_score", "author_position"],
    ascending=[True, False, True]
)

# Tambahkan kolom rank untuk setiap id_proposal
similarity_cosine_df_baru["rank"] = similarity_cosine_df_baru.groupby("id_proposal").cumcount() + 1

print(similarity_cosine_df_baru[["id_proposal", "mahasiswa","tahun_proposal","id_penelitian", "tahun_penelitian", "id_dosen", "dosen","selisih_tahun", "author_position","similarity_score","rank","beban"]])
similarity_cosine_df_baru.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/beban_cosine_topik_urut_author.csv", index=False)

     id_proposal mahasiswa  tahun_proposal id_penelitian  tahun_penelitian  \
0             P1        S1            2021           R48              2018   
142           P1        S1            2021           R48              2018   
143           P1        S1            2021          R186              2020   
144           P1        S1            2021          R291              2017   
145           P1        S1            2021          R264              2016   
...          ...       ...             ...           ...               ...   
2243         P99       S99            2024          R364              2023   
2244         P99       S99            2024          R377              2023   
2245         P99       S99            2024          R464              2022   
2246         P99       S99            2024          R247              2019   
2247         P99       S99            2024           R55              2021   

     id_dosen              dosen  selisih_tahun  author_positio

# Evaluasi Cosine

coba

## Evaluasi Baru

In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


def evaluate_ordered_recommendation_cosine(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df = evaluate_ordered_recommendation_cosine(rank_all_df, true_label_df)
print(result_df)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.288732              0.204225              0.084507   
1      5               0.434272              0.204225              0.084507   
2      7               0.556338              0.204225              0.084507   
3     10               0.713615              0.204225              0.084507   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0              0.077465                  0.848544  
1              0.077465                  0.759507  
2              0.077465                  0.687328  
3              0.077465                  0.588596  


In [51]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_cosine_3 = evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=3)
eval_per_proposal_cosine_5 = evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=5)
eval_per_proposal_cosine_7 = evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=7)
eval_per_proposal_cosine_10 = evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=10)

# eval_per_proposal_cosine_3.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/hasil_eval_3_kata.csv", index=False)
# eval_per_proposal_cosine_5.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/hasil_eval_5_kata.csv", index=False)
# eval_per_proposal_cosine_7.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/hasil_eval_7_kata.csv", index=False)
# eval_per_proposal_cosine_10.to_csv("/content/drive/MyDrive/Skripsi4/kata/cosine/hasil_eval_10_kata.csv", index=False)